In [1]:
from scipy.optimize import minimize, Bounds, LinearConstraint
from datetime import timedelta 
from dateutil.parser import parse
import numpy as np
from numpy.linalg import norm
import pandas as pd

ModuleNotFoundError: No module named 'scipy'

In [49]:
data=pd.read_csv("data/top25Raw.csv",header=[0,1],index_col=0)
data_close=data.swaplevel(axis=1).Close
data_close

,AAVE-USD,ADA-USD,AVAX-USD,BCH-USD,BNB-USD,BTC-USD,CHZ-USD,CRV-USD,DAI-USD,DKA-USD,...,LTC-USD,MATIC-USD,NEAR-USD,SHIB-USD,SOL-USD,TRX-USD,VIDT-USD,WBNB-USD,WETH-USD,XRP-USD
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-21,35.160358,0.106187,3.965959,258.514587,29.772354,12823.689453,0.010546,0.382909,1.008075,0.028196,...,53.045330,0.016383,0.701406,0.000000,1.890378,0.026721,0.366028,29.703623,392.537170,0.251615
2020-10-22,38.031651,0.110572,4.314421,266.472626,30.658192,12965.891602,0.010773,0.406706,1.008214,0.029148,...,54.267277,0.017404,0.749234,0.000000,1.955653,0.027042,0.379246,30.686083,414.364960,0.256807
2020-10-28,32.394081,0.098977,4.066606,268.489716,30.363506,13271.285156,0.009933,0.496183,1.009405,0.026101,...,55.742512,0.015086,0.699119,0.000000,1.490053,0.026852,0.338716,30.310003,388.456177,0.245865
2020-10-29,29.098333,0.095376,3.775312,267.056061,29.842478,13437.882812,0.009550,0.445006,1.011680,0.026391,...,54.765854,0.013877,0.690082,0.000000,1.428936,0.026490,0.336851,29.854565,386.319977,0.242295
2020-10-30,31.217823,0.093369,3.714938,261.828125,28.508411,13546.522461,0.009451,0.397217,1.007328,0.024658,...,53.835033,0.013910,0.631671,0.000000,1.441336,0.025626,0.374163,28.570316,384.153595,0.239184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-19,58.623249,0.328512,12.882173,104.819794,272.397675,16711.546875,0.244877,0.543496,1.000480,0.027530,...,63.812397,0.876095,1.747957,0.000009,12.851014,0.052162,0.020062,272.054047,1218.567627,0.384699
2022-11-20,56.173927,0.312307,12.196779,104.890907,264.235138,16291.832031,0.197668,0.512084,1.000297,0.026492,...,62.002327,0.807574,1.569242,0.000009,12.252833,0.050036,0.026375,264.069397,1142.148804,0.361945
2022-11-21,55.928955,0.304548,11.834380,103.249161,254.168259,15787.284180,0.185841,0.501019,1.000218,0.025509,...,61.373489,0.796284,1.559391,0.000008,11.846209,0.051408,0.025272,254.591415,1107.492676,0.364799


In [2]:
#Mean returns function
def mean_ret(data,length):
    return data.sum()/length

def monthdelta (date,delta):
    m,y = (date.mounth+delta)%12 , date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31, 29 if y%4==0 and not y%400==0 else 28,31,30,31,30,31,31,30,31,30,31][m-1])
    new_date = (date.replace(day=d,month=m, year=y))
    return parse(new_date.strftime('%Y-%m-%d'))

def windowGenerator (dataframe, lookback, horizon, step, cummulative = False):
    if cummulative:
        c = lookback
        step = horizon
        
    initial = min(dataframe.index)
    windows = []
    horizons = []

    while initial <= monthdelta(max(dataframe.index), -lookback):
        windowStart = initial
        windowEnd = monthdelta(windowStart, lookback)
        if cummulative:
            windowStart = min(dataframe.index)
            windowEnd = monthdelta(windowStart, c) + timedelta(days=1)
            c += horizon
        horizonStart = windowEnd + timedelta(days=1)
        horizonEnd = monthdelta(horizonStart, horizon)

        lookbackWindow = dataframe[windowStart:windowEnd]
        horizonWindow = dataframe[horizonStart:horizonEnd]

        windows.append(lookbackWindow)
        horizons.append(horizonWindow)

        initial = monthdelta(initial, step)

    return windows, horizons

In [3]:
def actual_return(actual_ret, weight):
    mean_return = mean_ret(actual_ret,actual_ret.shape[0])
    actual_cov=actual_ret.cov()

    portfolio_return = mean_ret.T.dot(weight)
    protfolio_cov = weight.T.dot(actual_cov).dot(weight)
    return portfolio_return,protfolio_cov

def optimisation(predicted_ret, actual_ret, lam1, lam2):
    mean_return = mean_ret(predicted_ret, predicted_ret.shape[0])
    predicted_covariance = predicted_ret.cov()
#Cost function
    def f(weight):
        return -(mean_return.T.dot(weight) - lam1*(weight.T.dot(predicted_covariance).dot(weight)) + lam2*norm(weight, ord=1))

    opt_bounds = Bounds(0, 1)

#Equality Constraints
    def h(weight):
        return sum(weight) - 1

#Constraints Dictionary
    cons = ({
        'type' : 'eq',
      'fun' : lambda weight: h(weight)
  })

#Solver
    sol = minimize(f,
                 x0 = np.ones(mean_return.shape[0]),
                 constraints = cons,
                 bounds = opt_bounds,
                 options = {'disp': False},
                 tol=10e-10)  

#Predicted Results
    weight = sol.x
    predicted_portfolio_ret = weight.dot(mean_return)
    portfolio_STD = weight.T.dot(predicted_covariance).dot(weight)
  
#Actual Results
    portfolio_actual_returns, portfolio_actual_variance = actual_return(actual_ret, weight)
    sharpe_ratio = portfolio_actual_returns/np.std(portfolio_actual_variance)

    ret_dict = {'weights' : weight,
              'predicted_returns' : predicted_portfolio_ret,
              'predicted_variance' : portfolio_STD,
              'actual_returns' : portfolio_actual_returns,
              'actual_variance' : portfolio_actual_variance,
              'sharpe_ratio': sharpe_ratio}
  
    return ret_dict                       